In [40]:
import pandas as pd
import numpy as np
import selenium 
from selenium.webdriver.chrome import options
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from datetime import datetime as dt
import re


build data scrapping bot to get all data from an mlb season 

data style: group by date, each game that occured, who played, and what was the score

stage 1. 
just get one string that we want

stage 2. 
get many strings we want from one page

stage 3. 
get many string from many pages

In [30]:
# input parameters
# season (year)

In [95]:

# initialize webdriver
driver = webdriver.Chrome()

# hide the browser from view
# driver.options.add_argument('--headless')
                          
# mlb season starts March 30th and ends July 11th
start = dt(2022, 3, 30)
end = dt(2022, 7, 11)

date = start.strftime("%Y-%m-%d")
link = "https://www.mlb.com/schedule/{}".format(date)
print(link)
driver.get(link)

# get value of the first item in the table
# div id="gridWrapper" >> includes three days starting at url date
# div data-mlb-test="individualGamesContainer" >> individual days, but all the games for that day. First one is url date
# div data-mlb-test="individualGamesContainerDesktop" >> individual games, 

soup = BeautifulSoup(driver.page_source, "html.parser")

texts = []

rows = soup.find('div', {'data-mlb-test': 'individualGamesContainer'}).find_all('a')

for x in rows:
    texts.append(x.get_text())

driver.quit()


https://www.mlb.com/schedule/2022-03-30


In [96]:
stats = []

all_team_names_with_space = ['WSH ', 'TEX ', 'BOS ', 'OAK ', 'CHC ', 'NYY ', 'KC ', 'CLE ', 'TOR ', 'DET ', 'TB ', 'ATL ', 'MIA ', 'PHI ', 'SEA ', 'HOU ', 'BAL ', 'MIN ', 'STL ', 'CWS ', 'MIL ', 'CIN ', 'SF ', 'COL ', 'PIT ', 'AZ ', 'SD ', 'NYM ', 'NL ', 'AL ', 'LAA ', 'LAD ']

print(len(all_team_names_with_space))

for x in texts:
    for y in all_team_names_with_space:
        if (x.startswith(y)):
            stats.append(x)

print(len(stats))

print(stats)


32
14
['STL 29, WSH 8', 'BAL 7, TB 6', 'MIN 9, PIT 4', 'DET 7, PHI 1', 'BOS 10, ATL 7', 'OAK 5, CIN 4', 'CHC 8, SEA 5', 'SF 9, KC 5', 'CWS 7, TEX 0', 'AZ 9, COL 2', 'SD 4, MIL 2', 'HOU 5, NYM 3', 'TOR 11, NYY 3', 'LAD 12, CLE 1']


In [115]:
tW = []
sW = []
tL = []
sL = []

for x in stats:
    z = re.sub(",", "", x)
    y = z.split()
    print(y)
    tW.append(y[0])
    sW.append(int(y[1]))
    tL.append(y[2])
    sL.append(int(y[3]))

y

['STL', '29', 'WSH', '8']
['BAL', '7', 'TB', '6']
['MIN', '9', 'PIT', '4']
['DET', '7', 'PHI', '1']
['BOS', '10', 'ATL', '7']
['OAK', '5', 'CIN', '4']
['CHC', '8', 'SEA', '5']
['SF', '9', 'KC', '5']
['CWS', '7', 'TEX', '0']
['AZ', '9', 'COL', '2']
['SD', '4', 'MIL', '2']
['HOU', '5', 'NYM', '3']
['TOR', '11', 'NYY', '3']
['LAD', '12', 'CLE', '1']


['LAD', '12', 'CLE', '1']

In [ ]:
# create dictionary objects for 
# team and rating

teams = ['WSH', 'TEX', 'BOS', 'OAK', 'CHC', 'NYY', 'KC', 'CLE', 'TOR', 'DET', 'TB', 'ATL', 'MIA', 'PHI', 'SEA', 'HOU', 'BAL', 'MIN', 'STL', 'CWS', 'MIL', 'CIN', 'SF', 'COL', 'PIT', 'AZ', 'SD', 'NYM', 'NL', 'AL', 'LAA', 'LAD']
data = pd.DataFrame( 
    {
        f"opponent ({date})": None,
        f"points scored ({date})": 0,
        f"rating ({date})": 0
    }, index = teams)

data



In [114]:
# {'Class': 3, 'Roll': 22, 'Name': 'Sakshi', 'Height': 160}


for x, y, z, s in zip(tW, sW, tL, sL):
    data.loc[x] = [z, y, +1]

for x, y, z, s in zip(tW, sW, tL, sL):
    data.loc[z] = [x, y, -1]


data

,opponent (2022-03-30),points scored (2022-03-30),rating (2022-03-30)
WSH,STL,29,-1
TEX,CWS,7,-1
BOS,ATL,10,1
OAK,CIN,5,1
CHC,SEA,8,1
NYY,TOR,11,-1
KC,SF,9,-1
CLE,LAD,12,-1
TOR,NYY,11,1
DET,PHI,7,1


In [122]:
teams = ['WSH', 'TEX', 'BOS', 'OAK', 'CHC', 'NYY', 'KC', 'CLE', 'TOR', 'DET', 'TB', 'ATL', 'MIA', 'PHI', 'SEA', 'HOU', 'BAL', 'MIN', 'STL', 'CWS', 'MIL', 'CIN', 'SF', 'COL', 'PIT', 'AZ', 'SD', 'NYM', 'NL', 'AL', 'LAA', 'LAD']

df = pd.DataFrame(index = teams)



df = pd.concat([df, data], axis=1, join='inner')

df

,opponent (2022-03-30),points scored (2022-03-30),rating (2022-03-30)
WSH,STL,29,-1
TEX,CWS,7,-1
BOS,ATL,10,1
OAK,CIN,5,1
CHC,SEA,8,1
NYY,TOR,11,-1
KC,SF,9,-1
CLE,LAD,12,-1
TOR,NYY,11,1
DET,PHI,7,1
